# SOTA Workflow Analysis: TTD-DR vs. Linear Refinement

This notebook analyzes the **Test-Time Denoising for Deep Research (TTD-DR)** workflow compared to standard iterative refinement.

**Objective**: Demonstrate how "Denoising" (Cross-Variant Critique) produces better results than simple "Self-Correction".

In [ ]:
import random

# Mock LLM to simulate behavior
class MockLLM:
    def generate(self, prompt, temperature=0.7):
        # Simulate slight hallucinations or errors based on temperature
        if temperature > 0.7:
            return "Draft: High volatility in facts. [Hallucination: Python 4.0 released]"
        else:
            return "Draft: Conservative facts."

    def critique(self, answer):
        if "Python 4.0" in answer:
            return "Error: Python 4.0 is not released."
        return "Looks okay."

    def synthesize(self, variants, critiques):
        return f"Final Answer: Synthesized from {len(variants)} variants. Corrected: Python 3.13 is latest."

llm = MockLLM()

## 1. Linear Refinement (Self-Correction)

Standard agents generate one draft, then critique it, then fix it.

In [ ]:
def linear_refinement():
    print("--- Linear Refinement ---")
    # Step 1: Draft
    draft = llm.generate("Status of Python", temperature=0.8)
    print(f"Draft 1: {draft}")
    
    # Step 2: Critique
    critique = llm.critique(draft)
    print(f"Critique: {critique}")
    
    # Step 3: Fix
    if "Error" in critique:
        final = "Draft: Corrected facts. Python 3.13 is latest."
    else:
        final = draft
    print(f"Final: {final}\n")

linear_refinement()

## 2. TTD-DR (Denoising Diffusion)

TTD-DR generates *multiple* noisy variants (high temperature) and uses them to denoising each other.

In [ ]:
def ttd_dr_workflow():
    print("--- TTD-DR Workflow ---")
    # Step 1: Generate Noisy Variants
    variants = []
    for i in range(3):
        # Use high temp to get diversity
        v = llm.generate("Status of Python", temperature=0.9)
        variants.append(f"Variant {i}: {v}")
        print(f"Generated {variants[-1]}")
    
    # Step 2: Cross-Critique (Denoising)
    # In real TTD-DR, this is an iterative loop.
    critiques = []
    for v in variants:
        # The critique sees the OTHER variants
        critiques.append(llm.critique(v))
    
    # Step 3: Synthesis
    final = llm.synthesize(variants, critiques)
    print(f"{final}")

ttd_dr_workflow()

## 3. Analysis

**Why TTD-DR is better for Research:**

1.  **Diversity**: If the model has a 20% chance of hallucinating a fact, generating 1 draft risks failing 20% of the time. Generating 5 drafts ensures likely at least 3-4 are correct.
2.  **Cross-Check**: When the model sees "Variant A says X" and "Variant B says Y", it is forced to reconcile the conflict (checking evidence), whereas in Linear Refinement it might just confirm its own bias.

**Implementation Note**: This requires `parallel` execution support in the graph (e.g. `Send` API in LangGraph).